In [1]:
import os

folder_pwd = os.getcwd()
os.chdir('..')
os.chdir('../resources')
resources_path = os.getcwd()
os.chdir(folder_pwd)

image_folder = "/test/Images"
csv_folder = '/test/c'

files = os.listdir(resources_path+csv_folder)
csv_file = []
for f in files:
    csv_file.append(f)

csv_file

['Batch10.csv - TEST.csv', 'Batch9.csv - TEST.csv', 'Batch11.csv TEST.csv']

In [2]:
%%time
from csv import reader
import pandas as pd
import numpy as np

def read_csv(file_name):
    return pd.read_csv(resources_path+csv_folder+'/'+file_name, names=['type','from_x','from_y','width', 'height', 'name', 'full_width', 'full_height'])

df = pd.DataFrame()
for f in csv_file:
    df = df.append(read_csv(f)).reset_index(drop=True)

CPU times: user 696 ms, sys: 200 ms, total: 896 ms
Wall time: 1.27 s


In [3]:
images_to_check =df['name'].unique()
images_to_check.shape

(370,)

In [4]:
%%time

import os, random, cv2, Classify
import numpy as np

# 1. Load models
classifiers = Classify.load_svm(resources_path + '/SVM')
print(classifiers)

/Users/codingC/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator SVC from version 0.24.1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


{'head': SVC(probability=True), 'spine': SVC(probability=True), 'leg': SVC(probability=True), 'arm': SVC(probability=True), 'foot': SVC(probability=True), 'hand': SVC(probability=True)}
CPU times: user 951 ms, sys: 473 ms, total: 1.42 s
Wall time: 2.35 s


In [8]:
%%time
results = []
for image in images_to_check[:3]:
    img = cv2.imread(resources_path+image_folder+'/'+image, 0)
    wanted_classifiers = ['head','spine']  # The keys you want
    wanted_classifiers = dict((k, classifiers[k]) for k in wanted_classifiers if k in classifiers)
    feat_image, result, pred = Classify.analyse_image(img, classifiers,  crop_thresh=0.2, blur_size=10, bright_strength=1, sharp_strength=1,
                                   square_step=True, windows=[(64, 64), (96, 96), (128, 128), (192, 192)], window_stride=64, pred_threshold = 0.99)
    results.append(result)

results = pd.concat(results)
results

CPU times: user 12.7 s, sys: 196 ms, total: 12.9 s
Wall time: 17.4 s


,x,y,window_size,window,hog,head,spine,leg,arm,foot,hand,highest,highest_pred
116,178,409,"(192, 192)","[[77, 75, 73, 69, 66, 63, 62, 59, 58, 57, 57, ...","[[0.061106061419745276, 0.03184590867349936, 0...",0.992502,0.015657,0.014556,0.008140,0.005803,0.000009,head,0.992502
116,304,454,"(128, 128)","[[44, 44, 45, 46, 46, 46, 47, 45, 44, 43, 42, ...","[[0.14585371762088017, 0.06322553615382834, 0....",0.000008,0.991127,0.006896,0.004556,0.003958,0.000001,spine,0.991127
